In [2]:
import scipy.io as sio
from scipy.fft import fft
import os,glob
import math
import numpy as np
import pandas as pd
import os,glob
import pyexcel
import signal_utll

In [3]:
df = pd.read_csv('../datasets/laser_ds.csv')
df.head()

,Age Group,Sex,Side,Sub,File
0,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...
1,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...
2,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...
3,Age_A,Male,Neck_L,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_L_A...
4,Age_A,Male,Neck_L,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_L_A...


In [4]:
Hz_Lthr, Hz_Hthr, ROI_Hz_Lthr, ROI_Hz_Hthr ,low_weight, high_weight, high_weight2, no_peaks= 0.4, 3.5, 0.8, 2.5, 0.01, 0.07, 0.08, 5

top_peaks_array=[]
secs=60 # 1 mins

for i, row in df.iterrows():
    X, Y, peaks_idx = signal_utll.find_top_peaks_with_uniform_weight_v2(row['File'], Hz_Lthr, Hz_Hthr, 
                                                                     ROI_Hz_Lthr, ROI_Hz_Hthr, 
                                                                     low_weight, high_weight, high_weight2, no_peaks)
    peaks_hr = [round(X[peak]*secs) for peak in peaks_idx]
    top_peaks_array.append(peaks_hr)

df['Peaks_Hr'] = top_peaks_array

In [5]:
df.head()

,Age Group,Sex,Side,Sub,File,Peaks_Hr
0,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...,"[51.0, 54.0, 60.0, 62.0, 108.0]"
1,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...,"[74.0, 80.0, 113.0, 132.0, 143.0]"
2,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...,"[72.0, 93.0, 99.0, 101.0, 137.0]"
3,Age_A,Male,Neck_L,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_L_A...,"[67.0, 94.0, 96.0, 148.0, 150.0]"
4,Age_A,Male,Neck_L,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_L_A...,"[61.0, 68.0, 100.0, 124.0, 149.0]"


In [6]:
grouped_df = df.groupby(by=["Age Group","Sex","Sub"])

In [7]:
rows = []
for name, group in grouped_df: 
    n_row = list(name)
    gr_peaks = []
    for i, row in group.iterrows():
            gr_peaks.append(row['Peaks_Hr'])
    gr_peaks_list = [item for sublist in gr_peaks for item in sublist]
    #max_peak_cluster_center = signal_utll.do_clusters_and_find_max_center(gr_peaks_list, 5)
    
    n_row.append(round(max_peak_cluster_center, 2))
    rows.append(n_row)
    

df_Peaks= pd.DataFrame(rows);
df_Peaks

,0,1,2,3
0,Age_A,Female,Sub 25,120.50
1,Age_A,Female,Sub 28,108.75
2,Age_A,Female,Sub 32,102.00
3,Age_A,Female,Sub 35,92.40
4,Age_A,Female,Sub 44,130.62
5,Age_A,Female,Sub 46,100.11
6,Age_A,Female,Sub 63,130.00
7,Age_A,Female,Sub 64,108.38
8,Age_A,Female,Sub 66,99.14
9,Age_A,Female,Sub 69,140.29


In [7]:
df_Peaks.columns = ["Age Group","Sex","Sub", "Heart Beat 0.8~2.5hz"]
df_Peaks.head()

,Age Group,Sex,Sub,Heart Beat 0.8~2.5hz
0,Age_A,Female,Sub 25,53.92
1,Age_A,Female,Sub 28,57.21
2,Age_A,Female,Sub 32,60.75
3,Age_A,Female,Sub 35,54.06
4,Age_A,Female,Sub 44,51.90


In [8]:
df_Peaks.to_csv('../datasets/ds_hr_0.8_2.5hz-v1.csv',index = False, header=True)

In [28]:
#https://www.medicinenet.com/what_is_a_good_heart_rate_for_my_age/article.htm